In [ ]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_datareader.data as web
from datetime import datetime
import cvxopt as opt
from cvxopt import blas, solvers
import os
%matplotlib inline
%env ALPHAVANTAGE_API_KEY = NTN3D6P6LIMZ6A2Q#alpha advantage api key

In [ ]:
#choose the tickers of the PTF
ptf = ['AAPL','MSFT','KO','JNJ','XOM']
ptf_df = pd.DataFrame()

In [ ]:
#donwload adj-close data from the alphaadvantage api
for stock in ptf:
    ptf_df[stock] = web.DataReader(stock, "av-daily-adjusted", 
                                        start=datetime(2015,1, 1), 
                                        end=datetime(2021, 1, 16),api_key=os.getenv('ALPHAVANTAGE_API_KEY'))['adjusted close']

In [ ]:
ptf_df.head(10)

In [ ]:
#calculate the cumulative returns
cum_returns_df = ptf_df.apply(lambda x: (x/x[0]))
cum_returns_df.head()

In [ ]:
#import plotly
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px

In [ ]:
fig = px.line(cum_returns_df, x=cum_returns_df.index, y=cum_returns_df.columns)
fig.show()

In [ ]:
#compute daily log returns
log_returns = pd.DataFrame()
for stocks in ptf_df:
    log_returns[stocks] = np.log(ptf_df[stocks] / ptf_df[stocks].shift(1)).dropna()
log_returns.head(10)

In [ ]:
fig = px.line(log_returns, x=log_returns.index, y=log_returns.columns)
fig.show()

In [ ]:
log_returns.describe()

In [ ]:
def rand_weights(n):
    ''' Produces n random weights that sum to 1 '''
    k = np.random.rand(n)
    return k / sum(k)

In [ ]:
def random_portfolio(returns):
    ''' 
    Returns the mean and standard deviation of returns for a random portfolio
    '''

    p = np.asmatrix(np.mean(returns, axis=1))
    w = np.asmatrix(rand_weights(returns.shape[0]))
    C = np.asmatrix(np.cov(returns))
    
    mu = w * p.T
    sigma = np.sqrt(w * C * w.T)
    
    # This recursion reduces outliers to keep plots pretty
    if sigma > 2:
        return random_portfolio(returns)
    return mu, sigma

In [ ]:
#return_vec is a numpy ndarray with n_assets and n_obs
#transform the dataframe with stocks as columns as numpy array
#results are stacked
n_portfolios = 500
means, stds = np.column_stack([
    random_portfolio(log_returns.transpose().to_numpy()) 
    for _ in range(n_portfolios)
])

In [ ]:
#create a df in order to handle data better
df_test = pd.DataFrame(columns=['means','stds'])
for i in range(500):
    df_test.loc[i] = [means[i][0], stds[i][0] ] 

In [ ]:
fig = px.scatter(x=df_test.stds.values, y=df_test.means.values)
fig.show()


In [ ]:
def optimal_portfolio(returns):
    n = len(returns)
    returns = np.asmatrix(returns)
    
    N = 100
    mus = [10**(5.0 * t/N - 1.0) for t in range(N)]
    
    # Convert to cvxopt matrices
    S = opt.matrix(np.cov(returns))
    pbar = opt.matrix(np.mean(returns, axis=1))
    
    # Create constraint matrices
    G = -opt.matrix(np.eye(n))   # negative n x n identity matrix
    h = opt.matrix(0.0, (n ,1))
    A = opt.matrix(1.0, (1, n))
    b = opt.matrix(1.0)
    
    # Calculate efficient frontier weights using quadratic programming
    portfolios = [solvers.qp(mu*S, -pbar, G, h, A, b)['x'] 
                  for mu in mus]
    ## CALCULATE RISKS AND RETURNS FOR FRONTIER
    returns = [blas.dot(pbar, x) for x in portfolios]
    risks = [np.sqrt(blas.dot(x, S*x)) for x in portfolios]
    ## CALCULATE THE 2ND DEGREE POLYNOMIAL OF THE FRONTIER CURVE
    m1 = np.polyfit(returns, risks, 2)
    x1 = np.sqrt(m1[2] / m1[0])
    # CALCULATE THE OPTIMAL PORTFOLIO
    wt = solvers.qp(opt.matrix(x1 * S), -pbar, G, h, A, b)['x']
    return np.asarray(wt), returns, risks

In [ ]:
#return_vec is a numpy ndarray with n_assets and n_obs
weights, returns, risks = optimal_portfolio(log_returns.transpose().to_numpy())

In [ ]:
fig = px.scatter(x=risks, y=returns)
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_test.stds.values, y=df_test.means.values, mode='markers',name='markers'))
fig.add_trace(go.Scatter(x=risks, y=returns, mode='lines',name='Efficient Frontier'))
fig.update_layout(
    title="Markowitz Portfolio Optimization",
    xaxis_title="Std",
    yaxis_title="Mean" )
fig.show()